# ML - Competition

## Importing Libraries

In [2]:
#Import your libraries
import pandas as pd
import numpy as np
import seaborn as sns
import math
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style="darkgrid")

# Loading the Data

In [3]:
cookies_test = pd.read_csv("/Users/timeisenmenger/Desktop/Ironhack /Week8/ML-Competition/New - Datasets/cookies_validate.csv")
cookies_test.head()

,id,sugar to flour ratio,sugar index,bake temp,chill time,calories,density,pH,grams baking soda,bake time,quality,butter type,weight,diameter,mixins,crunch factor,aesthetic appeal
0,14,0.62,19.25,400.0,41.0,172.0,1.0002,7.98,0.67,9.7,0,melted,16.6,7,"nuts, chocolate",1.85,3
1,26,0.35,1.00,520.0,35.0,146.0,0.9930,8.45,0.44,10.0,0,melted,13.8,7,chocolate,1.43,3
2,30,0.39,10.40,440.0,20.0,142.0,0.9974,8.20,0.53,10.0,0,melted,17.0,7,chocolate,1.57,3
3,52,0.33,1.10,570.0,21.0,82.0,0.9910,8.32,0.46,10.9,0,melted,12.4,7,"chocolate, oats",1.44,3
4,100,0.37,13.50,600.0,52.0,192.0,0.9975,8.00,0.44,9.1,0,melted,14.8,7,chocolate,1.51,3


## Handling the "mixins" column

In [6]:
# One Hot Encoding the mixins column to get numerical values for each mixin

In [7]:
# getting dummies for each mixin
# separating strings by "," to get each mixin
# problem of having multiple mixin columns for 1 feature
dummie_test = cookies_test['mixins'].str.get_dummies(sep=',')
dummie_test

,chocolate,oats,peanut butter,chocolate,nuts,peanut butter,raisins
0,1,0,0,0,1,0,0
1,0,0,0,1,0,0,0
2,0,0,0,1,0,0,0
3,0,1,0,1,0,0,0
4,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...
774,1,1,0,0,1,0,0
775,0,1,0,0,1,0,0
776,0,1,0,0,1,0,0
777,0,0,0,0,0,1,0


In [8]:
# manually renaming columns
dummie_test.columns = ["chocolate", "oats", "peanut butter","chocolate_2", "nuts", "peanut butter_2", "raisins"]

In [9]:
# Adding values of wanted columns together to get full amount of values
dummie_test["peanut butter"] = dummie_test["peanut butter"] + dummie_test["peanut butter_2"]
dummie_test["chocolate"] = dummie_test["chocolate"] + dummie_test["chocolate_2"]
dummie_test

,chocolate,oats,peanut butter,chocolate_2,nuts,peanut butter_2,raisins
0,1,0,0,0,1,0,0
1,1,0,0,1,0,0,0
2,1,0,0,1,0,0,0
3,1,1,0,1,0,0,0
4,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...
774,1,1,0,0,1,0,0
775,0,1,0,0,1,0,0
776,0,1,0,0,1,0,0
777,0,0,1,0,0,1,0


In [11]:
# Dropping columns we dont need anymore
dummie_test = dummie_test.drop(["chocolate_2","peanut butter_2"],axis=1)
dummie_test

,chocolate,oats,peanut butter,nuts,raisins
0,1,0,0,1,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,1,0,0,0
4,1,0,0,0,0
...,...,...,...,...,...
774,1,1,0,1,0
775,0,1,0,1,0
776,0,1,0,1,0
777,0,0,1,0,0


In [12]:
# Putting both dataframes together
cookies_test = pd.concat([cookies_test, dummie_test], axis=1)
cookies_test

,id,sugar to flour ratio,sugar index,bake temp,chill time,calories,density,pH,grams baking soda,bake time,...,weight,diameter,mixins,crunch factor,aesthetic appeal,chocolate,oats,peanut butter,nuts,raisins
0,14,0.62,19.25,400.0,41.0,172.0,1.00020,7.98,0.67,9.7,...,16.6,7,"nuts, chocolate",1.85,3,1,0,0,1,0
1,26,0.35,1.00,520.0,35.0,146.0,0.99300,8.45,0.44,10.0,...,13.8,7,chocolate,1.43,3,1,0,0,0,0
2,30,0.39,10.40,440.0,20.0,142.0,0.99740,8.20,0.53,10.0,...,17.0,7,chocolate,1.57,3,1,0,0,0,0
3,52,0.33,1.10,570.0,21.0,82.0,0.99100,8.32,0.46,10.9,...,12.4,7,"chocolate, oats",1.44,3,1,1,0,0,0
4,100,0.37,13.50,600.0,52.0,192.0,0.99750,8.00,0.44,9.1,...,14.8,7,chocolate,1.51,3,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
774,1442,0.02,1.90,780.0,18.0,30.0,0.99712,8.40,0.75,9.8,...,13.8,7,"nuts, oats, chocolate",1.01,3,1,1,0,1,0
775,1445,0.19,5.20,940.0,19.0,98.0,0.99713,8.16,0.52,9.6,...,14.8,7,"nuts, oats",1.68,3,0,1,0,1,0
776,1461,0.00,2.10,600.0,6.0,13.0,0.99664,8.59,0.61,10.0,...,12.4,7,"nuts, oats",1.38,3,0,1,0,1,0
777,1467,0.08,2.30,670.0,19.0,32.0,0.99648,8.52,0.57,11.0,...,13.4,7,peanut butter,1.63,3,0,0,1,0,0


In [13]:
# dropping mixins column since we dont need it anymore
cookies_test.drop(["mixins"],axis=1, inplace=True)
cookies_test

,id,sugar to flour ratio,sugar index,bake temp,chill time,calories,density,pH,grams baking soda,bake time,...,butter type,weight,diameter,crunch factor,aesthetic appeal,chocolate,oats,peanut butter,nuts,raisins
0,14,0.62,19.25,400.0,41.0,172.0,1.00020,7.98,0.67,9.7,...,melted,16.6,7,1.85,3,1,0,0,1,0
1,26,0.35,1.00,520.0,35.0,146.0,0.99300,8.45,0.44,10.0,...,melted,13.8,7,1.43,3,1,0,0,0,0
2,30,0.39,10.40,440.0,20.0,142.0,0.99740,8.20,0.53,10.0,...,melted,17.0,7,1.57,3,1,0,0,0,0
3,52,0.33,1.10,570.0,21.0,82.0,0.99100,8.32,0.46,10.9,...,melted,12.4,7,1.44,3,1,1,0,0,0
4,100,0.37,13.50,600.0,52.0,192.0,0.99750,8.00,0.44,9.1,...,melted,14.8,7,1.51,3,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
774,1442,0.02,1.90,780.0,18.0,30.0,0.99712,8.40,0.75,9.8,...,cubed,13.8,7,1.01,3,1,1,0,1,0
775,1445,0.19,5.20,940.0,19.0,98.0,0.99713,8.16,0.52,9.6,...,cubed,14.8,7,1.68,3,0,1,0,1,0
776,1461,0.00,2.10,600.0,6.0,13.0,0.99664,8.59,0.61,10.0,...,cubed,12.4,7,1.38,3,0,1,0,1,0
777,1467,0.08,2.30,670.0,19.0,32.0,0.99648,8.52,0.57,11.0,...,cubed,13.4,7,1.63,3,0,0,1,0,0


## Encoding the "butter type" column 

In [14]:
# Getting dummies for butter type column
# Only 2 possible values with "cubed" and "melted"
dummie2_test = cookies_test['butter type'].str.get_dummies()
dummie2_test

,cubed,melted
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1
...,...,...
774,1,0
775,1,0
776,1,0
777,1,0


In [15]:
# Again merging the 2 dataframes
cookies_test = pd.concat([cookies_test, dummie2_test], axis=1)
cookies_test

,id,sugar to flour ratio,sugar index,bake temp,chill time,calories,density,pH,grams baking soda,bake time,...,diameter,crunch factor,aesthetic appeal,chocolate,oats,peanut butter,nuts,raisins,cubed,melted
0,14,0.62,19.25,400.0,41.0,172.0,1.00020,7.98,0.67,9.7,...,7,1.85,3,1,0,0,1,0,0,1
1,26,0.35,1.00,520.0,35.0,146.0,0.99300,8.45,0.44,10.0,...,7,1.43,3,1,0,0,0,0,0,1
2,30,0.39,10.40,440.0,20.0,142.0,0.99740,8.20,0.53,10.0,...,7,1.57,3,1,0,0,0,0,0,1
3,52,0.33,1.10,570.0,21.0,82.0,0.99100,8.32,0.46,10.9,...,7,1.44,3,1,1,0,0,0,0,1
4,100,0.37,13.50,600.0,52.0,192.0,0.99750,8.00,0.44,9.1,...,7,1.51,3,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
774,1442,0.02,1.90,780.0,18.0,30.0,0.99712,8.40,0.75,9.8,...,7,1.01,3,1,1,0,1,0,1,0
775,1445,0.19,5.20,940.0,19.0,98.0,0.99713,8.16,0.52,9.6,...,7,1.68,3,0,1,0,1,0,1,0
776,1461,0.00,2.10,600.0,6.0,13.0,0.99664,8.59,0.61,10.0,...,7,1.38,3,0,1,0,1,0,1,0
777,1467,0.08,2.30,670.0,19.0,32.0,0.99648,8.52,0.57,11.0,...,7,1.63,3,0,0,1,0,0,1,0


In [16]:
# Dropping "butter type" column since we dont need it anymore
cookies_test.drop(["butter type"],axis=1, inplace=True)
cookies_test.head()

,id,sugar to flour ratio,sugar index,bake temp,chill time,calories,density,pH,grams baking soda,bake time,...,diameter,crunch factor,aesthetic appeal,chocolate,oats,peanut butter,nuts,raisins,cubed,melted
0,14,0.62,19.25,400.0,41.0,172.0,1.0002,7.98,0.67,9.7,...,7,1.85,3,1,0,0,1,0,0,1
1,26,0.35,1.00,520.0,35.0,146.0,0.9930,8.45,0.44,10.0,...,7,1.43,3,1,0,0,0,0,0,1
2,30,0.39,10.40,440.0,20.0,142.0,0.9974,8.20,0.53,10.0,...,7,1.57,3,1,0,0,0,0,0,1
3,52,0.33,1.10,570.0,21.0,82.0,0.9910,8.32,0.46,10.9,...,7,1.44,3,1,1,0,0,0,0,1
4,100,0.37,13.50,600.0,52.0,192.0,0.9975,8.00,0.44,9.1,...,7,1.51,3,1,0,0,0,0,0,1


## Handling NaN values

In [18]:
# Checking for NaN values
# we see, its only 15 rows out of 5200, so we drop them
cookies_test.isna().sum()

id                      0
sugar to flour ratio    0
sugar index             0
bake temp               0
chill time              0
calories                0
density                 0
pH                      0
grams baking soda       0
bake time               0
quality                 0
weight                  0
diameter                0
crunch factor           0
aesthetic appeal        0
chocolate               0
oats                    0
peanut butter           0
nuts                    0
raisins                 0
cubed                   0
melted                  0
dtype: int64

## Dropping columns we dont need

In [19]:
# Dropping "butter type" column since we dont need it anymore
cookies_test.drop(["diameter", "aesthetic appeal", "id"],axis=1, inplace=True)
cookies_test.head()

,sugar to flour ratio,sugar index,bake temp,chill time,calories,density,pH,grams baking soda,bake time,quality,weight,crunch factor,chocolate,oats,peanut butter,nuts,raisins,cubed,melted
0,0.62,19.25,400.0,41.0,172.0,1.0002,7.98,0.67,9.7,0,16.6,1.85,1,0,0,1,0,0,1
1,0.35,1.00,520.0,35.0,146.0,0.9930,8.45,0.44,10.0,0,13.8,1.43,1,0,0,0,0,0,1
2,0.39,10.40,440.0,20.0,142.0,0.9974,8.20,0.53,10.0,0,17.0,1.57,1,0,0,0,0,0,1
3,0.33,1.10,570.0,21.0,82.0,0.9910,8.32,0.46,10.9,0,12.4,1.44,1,1,0,0,0,0,1
4,0.37,13.50,600.0,52.0,192.0,0.9975,8.00,0.44,9.1,0,14.8,1.51,1,0,0,0,0,0,1


In [20]:
cookies_test.shape

(779, 19)

## Handling Outliers

In [21]:
from scipy import stats
cookies_test[(np.abs(stats.zscore(cookies_test)) < 3).all(axis=1)]


/Users/timeisenmenger/opt/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:2500: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


,sugar to flour ratio,sugar index,bake temp,chill time,calories,density,pH,grams baking soda,bake time,quality,weight,crunch factor,chocolate,oats,peanut butter,nuts,raisins,cubed,melted


In [22]:
cookies_test.shape

(779, 19)

## Making a second DataFrame without columns "density", "crunch factor", "pH"

In [24]:
cookies2_test = cookies_test.copy()

In [25]:
cookies2_test.drop(["density", "crunch factor", "pH"], axis = 1, inplace=True)
cookies2_test.head()

,sugar to flour ratio,sugar index,bake temp,chill time,calories,grams baking soda,bake time,quality,weight,chocolate,oats,peanut butter,nuts,raisins,cubed,melted
0,0.62,19.25,400.0,41.0,172.0,0.67,9.7,0,16.6,1,0,0,1,0,0,1
1,0.35,1.00,520.0,35.0,146.0,0.44,10.0,0,13.8,1,0,0,0,0,0,1
2,0.39,10.40,440.0,20.0,142.0,0.53,10.0,0,17.0,1,0,0,0,0,0,1
3,0.33,1.10,570.0,21.0,82.0,0.46,10.9,0,12.4,1,1,0,0,0,0,1
4,0.37,13.50,600.0,52.0,192.0,0.44,9.1,0,14.8,1,0,0,0,0,0,1


## Exporting the 2 DataFrames

In [26]:
cookies_test.to_csv(r'/Users/timeisenmenger/Desktop/Ironhack /Week8/ML-Competition/Modified-DataFrames/Cookies_test.csv', index = False)


In [27]:
cookies2_test.to_csv(r'/Users/timeisenmenger/Desktop/Ironhack /Week8/ML-Competition/Modified-DataFrames/Cookies2_test.csv', index = False)
